In [32]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [33]:
import pandas as pd
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import keras_tuner as kt

In [34]:
train_labels = []
train_samples = []

In [35]:
# read in the file
training_dataset = pd.read_csv(
    r'./waters_datasets/Cleaned_Datasets/Merged_Dataset_For_Trainging.csv',
    usecols=[
        "DO","PH","Conductivity","BOD","NI","Fec_col","Tot_col","WQI"
    ]
)[["DO","PH","Conductivity","BOD","NI","Fec_col","Tot_col","WQI"]]

x_train = training_dataset[["DO","PH","Conductivity","BOD","NI","Fec_col","Tot_col"]]
y_train = training_dataset["WQI"]

In [36]:
x_train.head()

,DO,PH,Conductivity,BOD,NI,Fec_col,Tot_col
0,6.6,6.5,748.0,1.4,7.0,18.0,12000.0
1,6.6,6.4,338.0,1.1,0.7,75.0,5500.0
2,6.6,7.0,333.0,1.4,0.5,0.0,91.0
3,6.6,5.7,791.0,1.9,0.3,0.0,46.0
4,6.6,7.0,510.0,2.2,3.1,54.0,3350.0


In [37]:
y_train.head()

0    46.167450
1    40.826550
2    51.027703
3    32.954697
4    60.342890
Name: WQI, dtype: float64

In [38]:
# Testing dataset 1
testing_dataset_1 = pd.read_csv(r'./waters_datasets/Cleaned_Datasets/testing_data_1.csv',
    usecols=[
        "DO","PH","Conductivity","BOD","NI","Fec_col","Tot_col","WQI",
    ]
)[["DO","PH","Conductivity","BOD","NI","Fec_col","Tot_col","WQI"]]

In [39]:
testing_dataset_1.head()

,DO,PH,Conductivity,BOD,NI,Fec_col,Tot_col,WQI
0,9.0,8.0,85.0,0.1,0.2,106.0,397.0,47.656576
1,9.0,8.0,102.0,0.3,0.4,153.0,954.0,50.613823
2,9.0,8.0,96.0,0.2,0.3,58.0,653.0,47.550650
3,9.0,8.0,94.0,0.2,0.4,34.0,317.0,46.956823
4,10.0,8.0,112.0,0.2,0.5,213.0,1072.0,46.282847


In [40]:
# testing cloumns for x 
x_test_1 = testing_dataset_1[["DO","PH","Conductivity","BOD","NI","Fec_col","Tot_col"]]

In [41]:
x_test_1.head()

,DO,PH,Conductivity,BOD,NI,Fec_col,Tot_col
0,9.0,8.0,85.0,0.1,0.2,106.0,397.0
1,9.0,8.0,102.0,0.3,0.4,153.0,954.0
2,9.0,8.0,96.0,0.2,0.3,58.0,653.0
3,9.0,8.0,94.0,0.2,0.4,34.0,317.0
4,10.0,8.0,112.0,0.2,0.5,213.0,1072.0


In [42]:
# testing cloumns for y
y_test_1 = y = testing_dataset_1["WQI"]

In [43]:
y_test_1.head()

0    47.656576
1    50.613823
2    47.550650
3    46.956823
4    46.282847
Name: WQI, dtype: float64

In [44]:
#Use the hp argument to define the hyperparameters during model creation.
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(
        hp.Choice('units', [8, 16, 32, 64, 128, 256, 512]),
        activation='relu'))
    model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32, 64, 128, 256, 512]),
      activation='relu'))
    model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32, 64, 128, 256, 512]),
      activation='relu'))
    model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32, 64, 128, 256,  512]),
      activation='relu'))
    model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32, 64, 128, 256, 512]),
      activation='relu'))
   
    
    
    model.add(keras.layers.Dense(1, activation='linear'))
    model.compile(loss='mse')
    return model

In [47]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Number of GPUs available: ",len(physical_devices))

Number of GPUs available:  0


In [48]:
#add hyperparameter tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20)


In [50]:
tuner.search(x_train, y_train, epochs=500, validation_data=(x_test_1, y_test_1))
model = tuner.get_best_models()[0]

Trial 2 Complete [00h 01m 43s]
val_loss: 45372.61328125

Best val_loss So Far: 45372.61328125
Total elapsed time: 00h 03m 45s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
units             |512               |8                 

Epoch 1/500
50/50 [==============================] - 1s 12ms/step - loss: 1416834318336.0000 - val_loss: 53198036.0000
Epoch 2/500
50/50 [==============================] - 0s 9ms/step - loss: 21363814400.0000 - val_loss: 518667328.0000
Epoch 3/500
50/50 [==============================] - 0s 9ms/step - loss: 72882266112.0000 - val_loss: 356540224.0000
Epoch 4/500
50/50 [==============================] - 0s 9ms/step - loss: 18832539648.0000 - val_loss: 5451532288.0000
Epoch 5/500
50/50 [==============================] - 0s 9ms/step - loss: 61098483712.0000 - val_loss: 415272672.0000
Epoch 6/500
50/50 [==============================] - 0s 9ms/step - loss: 2383454208.0000 - val_loss: 8145520128.0000
Epoch 7/500
50/50 [========

_NotOkStatusException: InvalidArgumentError: Error while reading OwnedIterator._type_spec.

In [ ]:
prediction = model.predict(x_test)